In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

In [8]:
df_prod = pd.read_csv("./raw_data/production-quantity_raw.csv")
df_prod.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,24,Angola,5312,Area harvested,1610,"Coffee, green",1961,1961,ha,350000.0,A,Official figure,NaN
1,QCL,Crops and livestock products,24,Angola,5510,Production,1610,"Coffee, green",1961,1961,t,168600.0,A,Official figure,NaN
2,QCL,Crops and livestock products,24,Angola,5312,Area harvested,1610,"Coffee, green",1962,1962,ha,500000.0,A,Official figure,NaN
3,QCL,Crops and livestock products,24,Angola,5510,Production,1610,"Coffee, green",1962,1962,t,185000.0,A,Official figure,NaN
4,QCL,Crops and livestock products,24,Angola,5312,Area harvested,1610,"Coffee, green",1963,1963,ha,500000.0,E,Estimated value,NaN


##### first data understandings - (PRODUCTION QUANTITY & AREA HARVESTED)

In [4]:
df_prod.shape

(10556, 15)

In [6]:
df_prod.dtypes

Domain Code          object
Domain               object
Area Code (M49)       int64
Area                 object
Element Code          int64
Element              object
Item Code (CPC)       int64
Item                 object
Year Code             int64
Year                  int64
Unit                 object
Value               float64
Flag                 object
Flag Description     object
Note                 object
dtype: object

In [9]:
#Columns to keep: Area Code (M49), Area, Element (make own columns for each element), Year, Unit, Value

In [10]:
df_prod.describe()

,Area Code (M49),Element Code,Item Code (CPC),Year Code,Year,Value
count,10556.000000,10556.000000,10556.0,10556.000000,10556.000000,1.018800e+04
mean,426.635468,5411.318871,1610.0,1991.807408,1991.807408,1.039288e+05
std,245.768005,99.004176,0.0,18.137591,18.137591,2.984428e+05
min,24.000000,5312.000000,1610.0,1961.000000,1961.000000,0.000000e+00
25%,192.000000,5312.000000,1610.0,1976.000000,1976.000000,5.400000e+02
50%,404.000000,5510.000000,1610.0,1992.000000,1992.000000,1.000000e+04
75%,630.000000,5510.000000,1610.0,2007.000000,2007.000000,7.804375e+04
max,894.000000,5510.000000,1610.0,2023.000000,2023.000000,4.462657e+06
